In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display
import librosa.display
import numpy as np
import librosa
import tensorflow as tf
import glob

C:\Users\Jaesung\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import glob
#uav_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_01_PHANTOMS/*.wav'
#none_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_0*.wav'
#uav_path = '../../../0720/*.wav'
#none_path = '../../../RW_AUDIO_DATA_2018_Update/JUNE_02_BACKGROUND/background_0*.wav'

uav_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/Sounddata_new/22050/Drone_22050/*.*'
none_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/Sounddata_new/22050/None_22050/*.*'

#uav_path ='C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/Sounddata_new/*.wav'

# C:\Users\Jaesung\Desktop\코딩\Python실습\K-SW-PJT\Sounddata_new
uav_files = glob.glob(uav_path)
none_files = glob.glob(none_path)
#print(uav_files)
#print(none_files)

In [3]:
CHUNK_SIZE = 8192
SR = 22050
N_MFCC = 16

In [4]:
def load(files, sr=22050):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
uav_raw = load(uav_files)
none_raw = load(none_files)

(35898530,)
(10518528,)


# Data preprocessing

Chose `mfcc4` among functions below:
- input size 16x16; `n_mfcc=16`, used first 16 data points from 16x17 mfcc of a chunk with size 8192
- chunk siez 8192, overlapped half of it

In [6]:
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=22050, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc, n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0], mfcc_slice.shape[1]))
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

In [7]:
mfcc_uav, y_uav = mfcc4(uav_raw, 1)
print(mfcc_uav.shape, y_uav.shape)
mfcc_none, y_none = mfcc4(none_raw, 0)
print(mfcc_none.shape, y_none.shape)

(35898530,)
small end: (16, 11)
small end: (16, 3)
(8763, 16, 16) (8763,)
(10518528,)
small end: (16, 9)
(2567, 16, 16) (2567,)


In [8]:
print(mfcc_uav.shape, y_uav.shape)
print(mfcc_none.shape, y_none.shape)

(8763, 16, 16) (8763,)
(2567, 16, 16) (2567,)


In [9]:
X = np.concatenate((mfcc_uav, mfcc_none), axis=0)
y = np.hstack((y_uav, y_none))
print(X.shape, y.shape)

(11330, 16, 16) (11330,)


In [10]:
X = np.reshape(X,(X.shape[0],-1))# 선범 

In [11]:
X.shape

(11330, 256)

In [12]:
n_labels = y.shape[0]
n_unique_labels = 2
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)

(11330, 2)


In [13]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [14]:
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [15]:
print(X_train.shape,y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(7251, 256) (7251, 2)
(1813, 256) (1813, 2)
(2266, 256) (2266, 2)


In [16]:
X_val.shape,y_val.shape

((1813, 256), (1813, 2))

In [17]:
### path selection between jaesung & gunhoo ###
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d', X_train)
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d', X_test)
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d', y_train)
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d', y_test)
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d', X_test)
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_val_chunk_3d', y_val)
np.save('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_val_chunk_3d', X_val)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d', X_train)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d', X_test)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d', y_train)
#np.save('../../../RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d', y_test)

In [18]:
### path selection between jaesung & gunhoo ###
X_train = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d.npy')
X_test = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d.npy')
y_train = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d.npy')
y_test = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d.npy')
y_val = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/y_val_chunk_3d.npy')
X_val = np.load('C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/RW_AUDIO_DATA_2018_Update/RW_AUDIO_DATA_2018_Update/Xy/X_val_chunk_3d.npy')

#X_train = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/X_train_chunk_3d.npy')
#X_test = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/X_test_chunk_3d.npy')
#y_train = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/y_train_chunk_3d.npy')
#y_test = np.load('../../../RW_AUDIO_DATA_2018_Update/Xy/y_test_chunk_3d.npy')

# Experiment 3 - One convolutional layer /w no dropout


##Experiment 3-2
- learning rate 0.005
- pooling stride 1x1
- #filter 1
- best result among every other settings
- cost kept fluctuated during training. (0.8 -> 1.3) -- why is that?

In [19]:
tf.reset_default_graph()

In [20]:
n_mfcc = 16
n_frame = 16
n_classes = 2
n_channels = 1

kernel_size = 3
stride = 1
pad = "SAME"

learning_rate = 0.0002  # 0.005
#0.0002 
training_epochs = 10

# Layer ver 2 - no cnn just Dense

## Experiment 3-2-변형
- learning rate 0.0002
- no cnn/ pooling 
- just dense with dropout 

In [21]:
X = tf.placeholder(tf.float32, shape=[None,n_mfcc*n_frame*n_channels])
#X = tf.reshape(X, [-1, n_mfcc, n_frame, n_channels]) #fully connected에는 리셰입 안해도 된다 
Y = tf.placeholder(tf.float32, shape=[None,n_classes])

keep_prob = tf.placeholder(tf.float32)

#dense2 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
#dropout2 = tf.layers.dropout(inputs=dense2, rate=0.5, training=True)

dense1 = tf.layers.dense(inputs=X, units=256, activation=tf.nn.relu)
dropout1 = tf.nn.dropout(dense1, keep_prob=keep_prob)


dense2 = tf.layers.dense(inputs=dropout1, units=256, activation=tf.nn.relu)
dropout2 = tf.nn.dropout(dense2, keep_prob=keep_prob)

dense3 = tf.layers.dense(inputs=dropout2, units=512, activation=tf.nn.relu)
dropout3 = tf.nn.dropout(dense3, keep_prob=keep_prob)

#이거 지워보고 돌려보고 
dense4 = tf.layers.dense(inputs=dropout3, units=512, activation=tf.nn.relu)
dropout4 = tf.nn.dropout(dense4, keep_prob=keep_prob)


dense5 = tf.layers.dense(inputs=dropout4, units=256, activation=tf.nn.relu)
dropout5 = tf.nn.dropout(dense5, keep_prob=keep_prob)


logits= tf.layers.dense(inputs=dropout5, units=2)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [22]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [23]:
#model_path = 'C:/Users/Jaesung/Desktop/코딩/Python실습/K-SW-PJT/GIt/uav-audio-detection/models/CNN/my_test_model_cnn'
model_path = '../models/CNN/my_test_model_cnn'
#saver = tf.train.Saver()
#model_path = './model_cnn.ckpt'
#model_path = '../my_test_model_cnn'
saver = tf.train.Saver()

In [24]:
import math

In [25]:
X_train.shape

(7251, 256)

In [26]:
X_val.shape

(1813, 256)

In [27]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
#    accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [28]:
from sklearn.metrics import accuracy_score

In [29]:
#original 
batch_size = 32
cost_history = np.empty(shape=[1], dtype=float)

for epoch in range(1,400):
    avg_cost = 0    
#    total_batch = int(X_train.shape[0] / batch_size)
    for i in range( int(math.ceil(len(X_train)/batch_size)) ): # 배치 사이즈로 나눈 나머지 다 돌 수 있게 
        x_ = X_train[batch_size * i: batch_size * (i + 1)]
        y_ = y_train[batch_size * i: batch_size * (i + 1)]
        feed_dict={X:x_, Y:y_, keep_prob:0.6}   # keep_prob 삭제 
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        cost_history = np.append(cost_history,cost)
        avg_cost += c
    #accuracy_val = sess.run([accuracy], feed_dict={X:X_val, Y:y_val
    val_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_val , keep_prob:1}) # keep_prob 삭제  
    val_true = sess.run(tf.argmax(y_val,1))
    accuracy_val = accuracy_score(val_pred, val_true)
    print('Epoch:', '%04d' % (epoch), 'cost = ', '{:.9f}'.format(avg_cost / len(X_train)), "\t검증 세트 정확도: {:.3f}%".format(accuracy_val * 100),)
saver.save(sess, model_path)

#if epoch% 20 == 0:
#   save

Epoch: 0001 cost =  0.268208764 	검증 세트 정확도: 92.554%
Epoch: 0002 cost =  0.039590701 	검증 세트 정확도: 97.794%
Epoch: 0003 cost =  0.016139467 	검증 세트 정확도: 98.235%
Epoch: 0004 cost =  0.011280963 	검증 세트 정확도: 98.290%
Epoch: 0005 cost =  0.006891348 	검증 세트 정확도: 99.117%
Epoch: 0006 cost =  0.005104209 	검증 세트 정확도: 99.228%
Epoch: 0007 cost =  0.004476769 	검증 세트 정확도: 99.117%
Epoch: 0008 cost =  0.002785059 	검증 세트 정확도: 99.007%
Epoch: 0009 cost =  0.003162237 	검증 세트 정확도: 99.228%
Epoch: 0010 cost =  0.002283875 	검증 세트 정확도: 99.173%
Epoch: 0011 cost =  0.001905002 	검증 세트 정확도: 99.117%
Epoch: 0012 cost =  0.001794167 	검증 세트 정확도: 99.228%
Epoch: 0013 cost =  0.001751522 	검증 세트 정확도: 99.228%
Epoch: 0014 cost =  0.001631128 	검증 세트 정확도: 99.448%
Epoch: 0015 cost =  0.001236246 	검증 세트 정확도: 99.283%
Epoch: 0016 cost =  0.001024138 	검증 세트 정확도: 99.173%
Epoch: 0017 cost =  0.001155951 	검증 세트 정확도: 99.338%
Epoch: 0018 cost =  0.001079431 	검증 세트 정확도: 99.393%
Epoch: 0019 cost =  0.001017852 	검증 세트 정확도: 99.559%
Epoch: 0020 

Epoch: 0159 cost =  0.000352439 	검증 세트 정확도: 99.614%
Epoch: 0160 cost =  0.000248318 	검증 세트 정확도: 99.504%
Epoch: 0161 cost =  0.000367252 	검증 세트 정확도: 99.504%
Epoch: 0162 cost =  0.000092679 	검증 세트 정확도: 99.614%
Epoch: 0163 cost =  0.000093594 	검증 세트 정확도: 99.504%
Epoch: 0164 cost =  0.000237648 	검증 세트 정확도: 99.559%
Epoch: 0165 cost =  0.000170301 	검증 세트 정확도: 99.559%
Epoch: 0166 cost =  0.000134574 	검증 세트 정확도: 99.448%
Epoch: 0167 cost =  0.000212054 	검증 세트 정확도: 99.559%
Epoch: 0168 cost =  0.000299069 	검증 세트 정확도: 99.559%
Epoch: 0169 cost =  0.000063711 	검증 세트 정확도: 99.504%
Epoch: 0170 cost =  0.000345568 	검증 세트 정확도: 99.504%
Epoch: 0171 cost =  0.000048856 	검증 세트 정확도: 99.504%
Epoch: 0172 cost =  0.000140238 	검증 세트 정확도: 99.559%
Epoch: 0173 cost =  0.000134375 	검증 세트 정확도: 99.504%
Epoch: 0174 cost =  0.000250021 	검증 세트 정확도: 99.614%
Epoch: 0175 cost =  0.000102983 	검증 세트 정확도: 99.504%
Epoch: 0176 cost =  0.000138450 	검증 세트 정확도: 99.559%
Epoch: 0177 cost =  0.000042453 	검증 세트 정확도: 99.559%
Epoch: 0178 

Epoch: 0317 cost =  0.000252567 	검증 세트 정확도: 99.614%
Epoch: 0318 cost =  0.000241383 	검증 세트 정확도: 99.559%
Epoch: 0319 cost =  0.000093525 	검증 세트 정확도: 99.669%
Epoch: 0320 cost =  0.000098671 	검증 세트 정확도: 99.669%
Epoch: 0321 cost =  0.000089298 	검증 세트 정확도: 99.559%
Epoch: 0322 cost =  0.000051155 	검증 세트 정확도: 99.614%
Epoch: 0323 cost =  0.000150402 	검증 세트 정확도: 99.614%
Epoch: 0324 cost =  0.000203871 	검증 세트 정확도: 99.559%
Epoch: 0325 cost =  0.000301880 	검증 세트 정확도: 99.614%
Epoch: 0326 cost =  0.000057612 	검증 세트 정확도: 99.559%
Epoch: 0327 cost =  0.000433072 	검증 세트 정확도: 99.504%
Epoch: 0328 cost =  0.000178287 	검증 세트 정확도: 99.504%
Epoch: 0329 cost =  0.000074988 	검증 세트 정확도: 99.559%
Epoch: 0330 cost =  0.000406623 	검증 세트 정확도: 99.393%
Epoch: 0331 cost =  0.000242144 	검증 세트 정확도: 99.448%
Epoch: 0332 cost =  0.000106488 	검증 세트 정확도: 99.504%
Epoch: 0333 cost =  0.000213116 	검증 세트 정확도: 99.228%
Epoch: 0334 cost =  0.000108310 	검증 세트 정확도: 99.504%
Epoch: 0335 cost =  0.000190668 	검증 세트 정확도: 99.338%
Epoch: 0336 

'../models/CNN/my_test_model_cnn'

In [30]:
y_pred = sess.run(tf.argmax(logits,1),feed_dict={X: X_test , keep_prob:1}) # keep_prob 삭제  
y_true = sess.run(tf.argmax(y_test,1))

In [31]:
from sklearn.metrics import precision_recall_fscore_support
p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
from sklearn.metrics import accuracy_score
print("Accuracy: ", accuracy_score(y_true, y_pred))

from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_true, y_pred))

F-Score: 0.994
Accuracy:  0.9942630185348632
             precision    recall  f1-score   support

          0       0.98      0.99      0.99       506
          1       1.00      1.00      1.00      1760

avg / total       0.99      0.99      0.99      2266

[[ 501    5]
 [   8 1752]]


In [39]:
from sklearn.metrics import accuracy_score